In [16]:
import gensim
import gensim.corpora as corpora
from gensim import models
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from nltk.stem.porter import *
stemmer = SnowballStemmer('english')
import numpy as np
np.random.seed(2018)
import nltk
import logging
import os
import requests as re
import pandas as pd
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
from pprint import pprint
import pyLDAvis.gensim
import pickle 
import pyLDAvis
from wordcloud import WordCloud
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /Users/kevins/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kevins/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/kevins/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [46]:
buzz_found = pd.read_csv(os.getcwd()+'/found_buzzwords_new.csv')

In [8]:
list(set(buzz_found['index'].tolist()))

{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188

In [21]:
papers = pd.read_csv(os.getcwd().replace('scripts', 'data/papers/corpus_papers.csv'))

In [22]:
papers

,journal_id,volume_id,year,link,doi,date,pii,eid,scopus_id,citations,authors,affiliations,open_access
0,1,0,2023,https://www.sciencedirect.com/science/article/...,10.1016/j.advwatres.2023.104474,2023-08-01,S0309170823001094,2-s2.0-85161258968,8.516126e+10,0.0,"58307014400, 36008901600, 26635952600",60031268:60031268:60007989,True
1,1,0,2023,https://www.sciencedirect.com/science/article/...,10.1016/j.advwatres.2023.104475,2023-08-01,S0309170823001100,2-s2.0-85161636300,8.516164e+10,0.0,"53984251100, 35610240400","60104134:60104134, 60022461",True
2,1,0,2023,https://www.sciencedirect.com/science/article/...,10.1016/j.advwatres.2023.104476,2023-08-01,S0309170823001112,2-s2.0-85161667219,8.516167e+10,0.0,25822520800,60025063,True
3,1,0,2023,https://www.sciencedirect.com/science/article/...,10.1016/j.advwatres.2023.104477,2023-08-01,S0309170823001124,2-s2.0-85161661769,8.516166e+10,0.0,"57210424267, 36160282300, 7404296426",60018394:60018394:60018394,True
4,1,0,2023,https://www.sciencedirect.com/science/article/...,10.1016/j.advwatres.2023.104485,2023-08-01,S0309170823001203,2-s2.0-85161678744,8.516168e+10,0.0,"56708113100, 7202660185",60030447:60007566,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25014,28,68,2018,https://www.sciencedirect.com/science/article/...,10.1016/j.envsoft.2017.09.020,2018-01-01,S1364815217304140,2-s2.0-85032268729,8.503227e+10,142.0,"34571084000, 55188785600, 33767605600, 5721638...","60016437:60012197:60012197:60012197, 60025278:...",True
25015,28,68,2018,https://www.sciencedirect.com/science/article/...,10.1016/j.envsoft.2017.09.010,2018-01-01,S1364815216304005,2-s2.0-85032353211,8.503235e+10,54.0,"37111876100, 15069126500, 57189996027, 3574567...","60013141:60025182:60025182:60025182:60013141, ...",True
25016,28,68,2018,https://www.sciencedirect.com/science/article/...,10.1016/j.envsoft.2017.08.002,2018-01-01,S1364815216309550,2-s2.0-85032684995,8.503268e+10,5.0,"7006615627, 49661600900, 15132591800, 66033150...","60021439:60030486, 109682148:60021439:60021439...",True
25017,28,68,2018,https://www.sciencedirect.com/science/article/...,10.1016/j.envsoft.2017.09.011,2018-01-01,S136481521630843X,2-s2.0-85032736133,8.503274e+10,45.0,"56111768300, 37010006200, 43361846100, 5611302...",60068861:60020071:60068861:60024695:60102538:6...,True


In [47]:
df = pd.read_csv(os.getcwd()+'/found_buzzwords_new.csv')

In [48]:
df

,index,doi,buzz_id,category,subword,count
0,16000,10.1016/j.watres.2022.118501,12,Data Analytics,data analysis,1
1,16001,10.1016/j.watres.2022.118494,0,AI and Machine Learning,machine learning,4
2,16001,10.1016/j.watres.2022.118494,0,AI and Machine Learning,classification,11
3,16001,10.1016/j.watres.2022.118494,0,AI and Machine Learning,support vector machine,2
4,16001,10.1016/j.watres.2022.118494,0,AI and Machine Learning,regression,11
...,...,...,...,...,...,...
893,16196,10.1016/j.watres.2022.118194,22,Smart Cities,circular economy,1
894,16198,10.1016/j.watres.2022.118182,7,Modeling,simulation,2
895,16198,10.1016/j.watres.2022.118182,14,Nanotechnology,nanotube,1
896,16198,10.1016/j.watres.2022.118182,14,Nanotechnology,carbon nanotube,1


In [ ]:
df[df['subword']=='risk assessment']

In [56]:
df[df['doi']=='10.1016/j.watres.2022.118418']

,index,doi,buzz_id,category,subword,count
47,16010,10.1016/j.watres.2022.118418,0,AI and Machine Learning,classification,3
48,16010,10.1016/j.watres.2022.118418,0,AI and Machine Learning,random forest,1
49,16010,10.1016/j.watres.2022.118418,0,AI and Machine Learning,regression,2
50,16010,10.1016/j.watres.2022.118418,0,AI and Machine Learning,neural network,4
51,16010,10.1016/j.watres.2022.118418,1,Big Data,data quality,2
52,16010,10.1016/j.watres.2022.118418,1,Big Data,data science,1
53,16010,10.1016/j.watres.2022.118418,3,IoT,real time monitoring,1
54,16010,10.1016/j.watres.2022.118418,7,Modeling,mike,2
55,16010,10.1016/j.watres.2022.118418,7,Modeling,spatial data,2
56,16010,10.1016/j.watres.2022.118418,7,Modeling,3d model,8


In [50]:
df.groupby('subword').count().sort_values('count', ascending = False).head(50)[['category','doi', 'count']]

,category,doi,count
subword,,,
statistical analysis,58,58,58
regression,50,50,50
simulation,48,48,48
classification,41,41,41
data analysis,36,36,36
nanoparticle,25,25,25
biotechnology,22,22,22
data processing,20,20,20
data visualization,20,20,20


In [34]:
df2[df2['count']>50]

,index,doi,buzz_id,category,count
subword,,,,,
api,1155579,10.1016/j.watres.2022.11850110.1016/j.watres.2...,2016,Web ApplicationsWeb ApplicationsWeb Applicatio...,227


In [28]:
df.loc[:]

,index,doi,buzz_id,category,subword,count
0,0,10.1016/j.advwatres.2023.104474,7,Modeling,simulation,24
1,0,10.1016/j.advwatres.2023.104474,7,Modeling,model validation,1
2,1,10.1016/j.advwatres.2023.104475,7,Modeling,simulation,21
3,1,10.1016/j.advwatres.2023.104475,7,Modeling,model parameters,1
4,1,10.1016/j.advwatres.2023.104475,15,Nanotechnology,nanoparticle,102
...,...,...,...,...,...,...
139241,26473,10.1016/j.envsoft.2017.09.002,7,Modeling,model development,3
139242,26473,10.1016/j.envsoft.2017.09.002,7,Modeling,model complexity,1
139243,26473,10.1016/j.envsoft.2017.09.002,29,Web Applications,user interface,2
139244,26473,10.1016/j.envsoft.2017.09.002,29,Web Applications,application programming interface,1


In [23]:
stop_words = stopwords.words('english')


In [24]:
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]

In [25]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [29]:
papers_to_process = pd.DataFrame(columns = ['index', 'text'])

# for i in range():
papers_to_process = pd.concat([papers_to_process, pd.read_csv('/Volumes/mac/corpus/text_corpus5.csv').loc[598:610]])

papers_to_process = papers_to_process.drop_duplicates().reset_index(drop = True)


In [30]:
papers_to_process

,index,text
0,8098,study region mozambique study focus mozambique...
1,8099,study region puakō hawai i island study focus ...
2,8100,study region the peace athabasca delta a ramsa...
3,8101,study region arogbo community ondo state niger...
4,8102,study region the upper part of the guayllabamb...
5,8103,study region northeast china study focus north...
6,8104,study region taiwan study focus the precipitat...
7,8105,study region andes central chile 32ºs 36ºs lak...
8,8106,study region the north china plain ncp is an i...
9,8107,study region peace athabasca delta pad northea...


In [31]:
processed_docs = papers_to_process['text'].map(preprocess)
id2word = corpora.Dictionary(processed_docs)
bow_corpus = [id2word.doc2bow(doc) for doc in processed_docs]

In [33]:
processed_docs

0     [studi, region, mozambiqu, studi, focus, mozam...
1     [studi, region, puakō, hawai, island, studi, f...
2     [studi, region, peac, athabasca, delta, ramsar...
3     [studi, region, arogbo, communiti, ondo, state...
4     [studi, region, upper, guayllabamba, napo, bas...
5     [studi, region, northeast, china, studi, focus...
6     [studi, region, taiwan, studi, focus, precipit...
7     [studi, region, andes, central, chile, lake, s...
8     [studi, region, north, china, plain, import, a...
9     [studi, region, peac, athabasca, delta, northe...
10    [studi, region, iran, studi, focus, grid, prec...
11    [studi, region, peatland, watersh, construct, ...
12    [studi, area, yellow, river, china, studi, foc...
Name: text, dtype: object

In [34]:
# number of topics
num_topics = 4
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=bow_corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[bow_corpus]

[(0,
  '0.030*"water" + 0.014*"lake" + 0.011*"level" + 0.008*"river" + '
  '0.008*"region" + 0.008*"precipit" + 0.008*"studi" + 0.006*"flood" + '
  '0.006*"resourc" + 0.006*"model"'),
 (1,
  '0.014*"water" + 0.013*"season" + 0.011*"lake" + 0.009*"precipit" + '
  '0.009*"sampl" + 0.008*"studi" + 0.008*"concentr" + 0.007*"differ" + '
  '0.006*"valu" + 0.005*"data"'),
 (2,
  '0.012*"water" + 0.010*"precipit" + 0.008*"region" + 0.008*"flood" + '
  '0.008*"studi" + 0.007*"lake" + 0.007*"sampl" + 0.007*"time" + '
  '0.006*"drought" + 0.005*"valu"'),
 (3,
  '0.019*"water" + 0.017*"lake" + 0.009*"sampl" + 0.008*"level" + '
  '0.008*"studi" + 0.007*"river" + 0.006*"concentr" + 0.006*"precipit" + '
  '0.006*"flood" + 0.006*"season"')]


In [35]:
pyLDAvis.enable_notebook()

In [36]:
LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))

In [17]:
#bag of words
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, bow_corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.021886  0.001347       1        1  36.341163
3      0.001151 -0.006634       2        1  24.421049
2     -0.008865  0.009998       3        1  23.718701
1     -0.014172 -0.004712       4        1  15.519087, topic_info=          Term        Freq       Total Category  logprob  loglift
659     season  279.000000  279.000000  Default  30.0000  30.0000
1350     sampl  256.000000  256.000000  Default  29.0000  29.0000
2176     metal  144.000000  144.000000  Default  28.0000  28.0000
963   concentr  228.000000  228.000000  Default  27.0000  27.0000
1717      lake  534.000000  534.000000  Default  26.0000  26.0000
...        ...         ...         ...      ...      ...      ...
677       show   24.613388  172.810478   Topic4  -5.6049  -0.0858
607     region   28.493224  292.069446   Topic4  -5.4585  -0.4642
638      river   27.445122  269.068756   Topic4  -5.4960  -0.4197
402      level   26.684334  328.809395   Topic4  -5.5241  -0.6483
535     period   22.114638  178.885410   Topic4  -5.7120  -0.2274

[337 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
10        1  0.247493  action
10        2  0.412489  action
10        3  0.247493  action
10        4  0.164996  action
3827      1  0.717769   alloc
...     ...       ...     ...
3936      4  0.065894    yrcc
3937      1  0.593490    yuan
3937      2  0.222559    yuan
3937      3  0.074186    yuan
3937      4  0.074186    yuan

[856 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 3, 2])

In [27]:
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.009394593513984821),
 (1, 0.01405563397152506),
 (2, 0.002081979111489905),
 (3, 0.0018450277753158883),
 (4, 0.014908928933304614),
 (5, 0.002981785786660923),
 (6, 0.014573853780429333),
 (7, 0.002081979111489905),
 (8, 0.00416395822297981),
 (9, 0.002981785786660923),
 (10, 0.00416395822297981),
 (11, 0.005348562314883217),
 (12, 0.002081979111489905),
 (13, 0.0320913738892993),
 (14, 0.01344062471308643),
 (15, 0.002981785786660923),
 (16, 0.01344062471308643),
 (17, 0.005348562314883217),
 (18, 0.008945357359982769),
 (19, 0.012491874668939428),
 (20, 0.01344062471308643),
 (21, 0.002081979111489905),
 (22, 0.037578374055939286),
 (24, 0.0006150092584386293),
 (25, 0.0013025311157816102),
 (26, 0.00416395822297981),
 (27, 0.0024600370337545174),
 (28, 0.026742811574416082),
 (29, 0.0018450277753158883),
 (30, 0.021083450957287586),
 (31, 0.01344062471308643),
 (32, 0.01344062471308643),
 (33, 0.002081979111489905),
 (34, 0.01405563397152506),
 (36, 0.002981785786660923),
 (

In [30]:
#TF IDF
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=3, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.001*"connector" + 0.001*"modul" + 0.001*"stiff" + 0.001*"modular" + 0.001*"float" + 0.001*"flexibl" + 0.001*"wave" + 0.000*"porous" + 0.000*"pipelin" + 0.000*"multi"
Topic: 1 Word: 0.001*"pile" + 0.001*"collis" + 0.001*"ship" + 0.001*"wave" + 0.001*"wake" + 0.001*"risk" + 0.001*"pipelin" + 0.001*"pipe" + 0.001*"turbin" + 0.001*"nonlinear"
Topic: 2 Word: 0.001*"porous" + 0.001*"column" + 0.001*"blast" + 0.001*"submerg" + 0.001*"explos" + 0.001*"concret" + 0.001*"wave" + 0.001*"salp" + 0.001*"pipelin" + 0.001*"benchmark"


In [35]:
#tfidf
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model_tfidf, bow_corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.001014 -0.000115       1        1  60.477307
2      0.000785 -0.000258       2        1  29.632778
0      0.000229  0.000374       3        1   9.889915, topic_info=           Term       Freq      Total Category  logprob  loglift
1495  connector  13.000000  13.000000  Default  30.0000  30.0000
1575      modul  12.000000  12.000000  Default  29.0000  29.0000
1426      stiff  12.000000  12.000000  Default  28.0000  28.0000
2328     porous  16.000000  16.000000  Default  27.0000  27.0000
2342    submerg  15.000000  15.000000  Default  26.0000  26.0000
...         ...        ...        ...      ...      ...      ...
1364    pipelin   1.411730  21.800732   Topic3  -7.6693  -0.4235
273        load   1.263944  13.117843   Topic3  -7.7798  -0.0261
788      propel   1.294622  17.433055   Topic3  -7.7559  -0.2865
1787       fuel   1.285988  16.963635   Topic3  -7.7626  -0.2659
1892    optimis   1.289848  17.844881   Topic3  -7.7596  -0.3135

[142 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
1472      1  0.626129    adjac
1472      2  0.268341    adjac
1472      3  0.089447    adjac
1480      1  0.611623   assign
1480      2  0.262124   assign
...     ...       ...      ...
2027      2  0.196679  weather
2027      3  0.065560  weather
897       1  0.610796   weight
897       2  0.305398   weight
897       3  0.076349   weight

[306 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1])

In [55]:
f_texts = pd.read_csv(os.getcwd().replace('scripts', 'data/text_corpus/text_corpus0.csv'))
                      
for i in range(1,100):
    f_texts = pd.concat([f_texts, pd.read_csv(os.getcwd().replace('scripts', f'data/text_corpus/text_corpus{i}.csv'))])

In [56]:
LDAvis_prepared = None

In [57]:
f_texts = f_texts.drop_duplicates().reset_index(drop = True)

In [59]:
f_texts

,index,text
0,0,although the fluid invasion process impacts th...
1,1,in order to investigate the migration of nanop...
2,2,in august 2019 this journal published the pape...
3,3,reservoir management and contaminant transport...
4,4,this study introduces firm information gain fo...
...,...,...
495,495,in large scale heterogeneous aquifer simulatio...
496,496,a new generation of smart stormwater systems p...
497,497,accurate modeling of variably saturated flow v...
498,498,stray gas migration resulting from oil and gas...


In [58]:
def getLDAbow(num_topics, threshold):
    processed_docs = f_texts['text'].map(preprocess)
    id2word = corpora.Dictionary(processed_docs)
    bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
    
    
    lda_model = gensim.models.LdaMulticore(corpus=bow_corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
    
    LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))
    
    
    #bag of words
    if 1 == 1:
        LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, bow_corpus, id2word)
        with open(LDAvis_data_filepath, 'wb') as f:
            pickle.dump(LDAvis_prepared, f)
    # load the pre-prepared pyLDAvis data from disk
    with open(LDAvis_data_filepath, 'rb') as f:
        LDAvis_prepared = pickle.load(f)
    pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')
    
    
#     # Print the Keyword in the 10 topics
#     pprint(lda_model.print_topics())
#     doc_lda = lda_model[bow_corpus]
    
    
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=tweets, dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print('nCoherence Score: ', coherence_lda)



In [ ]:
hi

In [85]:
threshold = 25

In [86]:
for i in range(26474):
    tmp = buzzwords_in_papers[buzzwords_in_papers['paper_id']==i]
    
    grouped = tmp.groupby('buzz_id').count()
    
#     print(grouped)
    
    for j in range(grouped.shape[0]):
        if grouped.iat[j,0] >=threshold:
            lots_of_words.loc[len(lots_of_words.index)] = [i, grouped.iloc[j].name, grouped.iat[j,0]]

In [74]:
lots_of_words

,paper_id,buzz_id,count
0,1,18,102
1,3,6,65
2,7,6,59
3,9,37,62
4,19,0,99
...,...,...,...
3441,26466,24,51
3442,26468,10,75
3443,26470,0,90
3444,26470,6,83
